# 03. EDA & InsightsObjective: Feature engineering, clustering for patterns, anomaly detection, and trend analysis.

In [ ]:
import pandas as pdimport numpy as npfrom sklearn.preprocessing import StandardScalerfrom sklearn.cluster import KMeansimport plotly.express as pximport plotly.graph_objects as goimport plotly.io as piopio.templates.default = 'plotly_white'

## 1. Load Processed Data

In [ ]:
master_df = pd.read_csv('processed_master_data.csv')master_df['date'] = pd.to_datetime(master_df['date'])print(f"Loaded Master Data: {master_df.shape}")

## 2. Feature Engineering

In [ ]:
# Define column groups (ensure lowercase matches cleaned data)cols_enc = ['age_0_5', 'age_5_17', 'age_18_greater']cols_bio = ['bio_age_5_17', 'bio_age_17_']cols_demo = ['demo_age_5_17', 'demo_age_17_']# 1. Total Enrolmentsmaster_df['new_enrolments'] = master_df[cols_enc].sum(axis=1)# 2. Total Updatesmaster_df['total_updates'] = master_df[cols_bio].sum(axis=1) + master_df[cols_demo].sum(axis=1)display(master_df[['state', 'date', 'new_enrolments', 'total_updates']].head())

## 3. Clustering Analysis (K-Means)Indentify state-level operational patterns.

In [ ]:
# Aggregating for state profilestate_profile = master_df.groupby('state').agg({    'age_0_5': 'sum',    'new_enrolments': 'sum',    'total_updates': 'sum'}).reset_index()# Ratiosstate_profile['ratio_infant'] = state_profile['age_0_5'] / state_profile['new_enrolments']state_profile['update_pressure'] = state_profile['total_updates'] / state_profile['new_enrolments']state_profile = state_profile.fillna(0).replace([np.inf, -np.inf], 0)# Clusteringscaler = StandardScaler()X = state_profile[['ratio_infant', 'update_pressure']]X_scaled = scaler.fit_transform(X)kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)state_profile['cluster_id'] = kmeans.fit_predict(X_scaled).astype(str)# Visualizationfig = px.scatter(    state_profile,    x='ratio_infant', y='update_pressure',    color='cluster_id', size='new_enrolments', hover_name='state',    title='State Operational Clusters',    labels={'ratio_infant': 'Infant Enrolment Ratio', 'update_pressure': 'Update Pressure'})fig.show()

## 4. Anomaly DetectionDetect persistent anomalies using 7-day rolling statistics.

In [ ]:
anomalies = []
WINDOW = 7
SIGMA = 3for state in master_df['state'].unique():    subset = master_df[master_df['state'] == state].sort_values('date').set_index('date')    ts = subset['new_enrolments']        if len(ts) < 10: continue        roll_mean = ts.rolling(WINDOW, min_periods=1).mean()    roll_std = ts.rolling(WINDOW, min_periods=1).std()        upper = roll_mean + (SIGMA * roll_std)    lower = roll_mean - (SIGMA * roll_std)        is_outlier = (ts > upper) | (ts < lower)        # Persistence check: 4 out of 5 rolling window    persistence = is_outlier.astype(int).rolling(window=5, center=True).sum()    persistent_outliers = is_outlier & (persistence >= 4)        if persistent_outliers.any():        for dt, val in ts[persistent_outliers].items():            anomalies.append({'state': state, 'date': dt, 'volume': val, 'expected': roll_mean.loc[dt]})            if anomalies:    print(f"Found {len(anomalies)} anomalies.")    display(pd.DataFrame(anomalies).head())else:    print("No persistent anomalies detected.")

## 5. Trend Analysis

In [ ]:
# National Daily Trendnational = master_df.groupby('date')[['new_enrolments', 'total_updates']].sum().reset_index()fig = go.Figure()fig.add_trace(go.Scatter(x=national['date'], y=national['new_enrolments'], name='New Enrolments'))fig.add_trace(go.Scatter(x=national['date'], y=national['total_updates'], name='Total Updates'))fig.update_layout(title='National Daily Volume Trends', xaxis_title='Date', yaxis_title='Volume')fig.show()

In [ ]:
# Age Cohort Analysiscohorts = pd.DataFrame({    'Cohort': ['0-5 Years', '5-17 Years', '18+ Years'],    'Volume': [        master_df['age_0_5'].sum(),        master_df['age_5_17'].sum(),        master_df['age_18_greater'].sum()    ]})px.bar(cohorts, x='Cohort', y='Volume', title='Total Enrolment by Age Group').show()